In [1]:
# Importing Necessory Packages:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Loading the Dataset
#%%time 
df = pd.read_csv('IMDB-Dataset.csv',encoding='latin-1')

In [8]:
pd.options.display.max_colwidth = 1000
df.head(5)

,Ratings,Reviews,Movies,Resenhas
0,1.0,"*Disclaimer: I only watched this movie as a conditional agreement. And I see films for free. I wouldn't be caught dead giving my hard earned money to these idiots.Well, to explain the depth of this 'film', I could write my shortest review, ever. Don't see this movie. It is by far the stupidest, lamest, most lazy, and unbelievably UNFUNNY movie I have ever seen. It is a total disaster. But since my hatred for this movie, and the others like it, extends far beyond one viewing, I think I'll go on for a bit.I don't know any of the people in the movie besides Carmen Electra, Vanessa Minnillo, and Kim Kardashian, but it doesn't matter. They're all horrible, though I think that was the point. The editing is flat out horrible, and possibly blatant continuity errors make this crapfast even crappier than I thought it would be. Now I know that these films are not supposed to be serious at all, but come on, it's film-making 101 that if someone gets a minor facial cut, it should be there in the...",Disaster Movie,"* IsenÃ§Ã£o de responsabilidade: eu sÃ³ assisti esse filme como um acordo condicional. E eu vejo filmes de graÃ§a. Eu nÃ£o seria pego morto dando meu dinheiro suado a esses idiotas. Bem, para explicar a profundidade desse 'filme', eu poderia escrever minha crÃ­tica mais curta de todos os tempos. NÃ£o vÃª este filme. Ã de longe o filme mais estÃºpido, lamenta, preguiÃ§oso e inacreditavelmente UNFUNNY que eu jÃ¡ vi. Ã um desastre total. Mas como o meu Ã³dio por este filme e por outros, se estende muito alÃ©m de uma exibiÃ§Ã£o, acho que vou continuar um pouco. NÃ£o conheÃ§o nenhuma das pessoas do filme alÃ©m de Carmen Electra, Vanessa Minnillo, e Kim Kardashian, mas isso nÃ£o importa. Eles sÃ£o todos horrÃ­veis, embora eu ache que esse seja o ponto. A ediÃ§Ã£o Ã© horrÃ­vel e, possivelmente, erros de continuidade flagrantes tornam essa porcaria ainda mais horrÃ­vel do que eu pensava. Agora eu sei que esses filmes nÃ£o devem ser sÃ©rios, mas vamos lÃ¡, Ã© o cinema 101 que se alguÃ©m f..."
1,1.0,"I am writing this in hopes that this gets put over the previous review of this ""film"". How anyone can find this slop entertaining is completely beyond me. First of all a spoof film entitled ""Disaster Movie"", should indeed be a spoof on disaster films. Now I have seen 1 (yes count them, 1) disaster film being spoofed, that being ""Twister"". How does Juno, Iron Man, Batman, The Hulk, Alvin and the Chipmunks, Amy Winehouse, or Hancock register as Disaster films? Selzterwater and Failburg once again have shown that they lack any sort of writing skill and humor. Having unfortunately been tortured with Date Movie and Epic Movie I know exactly what to expect from these two...no plot, no jokes just bad references and cheaply remade scenes from other films. Someone should have informed them that satire is more than just copy and paste from one film to another, though I shouldn't say that because some of these actually just seem to be taken from trailers.There is nothing clever or witty or re...",Disaster Movie,"Estou escrevendo isso na esperanÃ§a de que isso seja colocado sobre a revisÃ£o anterior deste ""filme"". Como alguÃ©m pode achar divertido esse desleixo estÃ¡ completamente alÃ©m de mim. Antes de mais nada, um filme de parÃ³dia intitulado ""Filme de desastre"" deveria ser, de fato, uma parÃ³dia de filmes de desastre. Agora eu jÃ¡ vi 1 (sim, conte-os, 1) filme de desastre sendo falsificado, sendo ""Twister"". Como Juno, Homem de Ferro, Batman, O Hulk, Alvin e os Esquilos, Amy Winehouse ou Hancock se registram como filmes de Desastre? Selzterwater e Failburg mostraram mais uma vez que nÃ£o possuem nenhum tipo de habilidade e humor de escrita. Infelizmente, tendo sido torturado com Date Movie e Epic Movie, sei exatamente o que esperar desses dois ... nenhum enredo, nenhuma piada, apenas mÃ¡s referÃªncias e cenas refeitas de outros filmes. AlguÃ©m deveria ter informado a eles que a sÃ¡tira Ã© mais do qu

In [9]:
df.shape

(150000, 4)

In [10]:
df['Ratings'].value_counts()

Ratings
1.0     15000
2.0     15000
4.0     15000
3.0     15000
5.0     15000
6.0     15000
8.0     15000
7.0     15000
10.0    15000
9.0     15000
Name: count, dtype: int64

In [ ]:
# It is clear that class is equally distributed.

## Data Cleaning and Preprocessing:

In [13]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [11]:
#Customize stopword as per data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
new_stopwords = ["would","shall","could","might"] # nltk didnt has these stopwords predefined so we hv to add
stop_words.extend(new_stopwords)
stop_words.remove("not")  # To check the sarcasm, multi polarity reviews
stop_words=set(stop_words)
print(stop_words)

{'he', 'over', 'such', 'isn', 'why', 'doing', 'nor', 't', 'how', 'could', "you're", 'herself', 'after', 'off', 'myself', 'shan', 'might', 'there', 'here', 'some', 'against', 'you', "needn't", 'its', 'his', 'ain', 'itself', 'a', 'me', "doesn't", "wasn't", "weren't", 'with', 'then', 'out', "she's", 'while', 'of', 'too', 'for', 're', 'haven', "shan't", 'your', 'who', 'on', 'don', "hasn't", 'other', 'wouldn', 'down', 'didn', 'him', 'at', 'just', 'up', 'through', 'now', "won't", "shouldn't", 'again', 'which', 'my', 'yourself', 'm', 'wasn', 'an', "wouldn't", "don't", 'do', 'shall', "isn't", 'had', 'i', 'where', 'am', 'themselves', 'all', "you've", 'the', 'be', 'this', 'being', 'as', 'few', 'most', 'both', 's', 'has', "aren't", 'before', 'once', 'only', 'so', 'under', 'did', 'during', "you'd", 'ourselves', 'these', 'that', 'no', 'if', 'hasn', 'will', 'mustn', "haven't", "didn't", 'mightn', 'ours', 'should', 'and', 'very', 'to', 'ma', 'does', 'himself', 'what', 'it', 'having', 'but', 'until', 

In [12]:
#Removing special character
def remove_special_character(content):
    return re.sub('\W+',' ', content )#re.sub('\[[^&@#!]]*\]', '', content)

# Removing URL's
def remove_url(content):
    return re.sub(r'http\S+', '', content)

#Removing the stopwords from text
def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

# Expansion of english contractions
def contraction_expansion(content):
    content = re.sub(r"won\'t", "would not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"don\'t", "do not", content)
    content = re.sub(r"shouldn\'t", "should not", content)
    content = re.sub(r"needn\'t", "need not", content)
    content = re.sub(r"hasn\'t", "has not", content)
    content = re.sub(r"haven\'t", "have not", content)
    content = re.sub(r"weren\'t", "were not", content)
    content = re.sub(r"mightn\'t", "might not", content)
    content = re.sub(r"didn\'t", "did not", content)
    content = re.sub(r"n\'t", " not", content)
    '''content = re.sub(r"\'re", " are", content)
    content = re.sub(r"\'s", " is", content)
    content = re.sub(r"\'d", " would", content)
    content = re.sub(r"\'ll", " will", content)
    content = re.sub(r"\'t", " not", content)
    content = re.sub(r"\'ve", " have", content)
    content = re.sub(r"\'m", " am", content)'''
    return content

In [13]:
#Data preprocessing
def data_cleaning(content):
    content = contraction_expansion(content)
    content = remove_special_character(content)
    content = remove_url(content)
    
    content = remove_stopwords(content)    
    return content

In [14]:
%%time
#Data cleaning
df['Reviews_clean']=df['Reviews'].apply(data_cleaning)

CPU times: total: 24.8 s
Wall time: 26 s


In [39]:
#pd.options.display.max_colwidth = 1000
df.head(5)

,Ratings,Reviews,Movies,Resenhas,Reviews_clean
0,1.0,"*Disclaimer: I only watched this movie as a conditional agreement. And I see films for free. I wouldn't be caught dead giving my hard earned money to these idiots.Well, to explain the depth of this 'film', I could write my shortest review, ever. Don't see this movie. It is by far the stupidest, lamest, most lazy, and unbelievably UNFUNNY movie I have ever seen. It is a total disaster. But since my hatred for this movie, and the others like it, extends far beyond one viewing, I think I'll go on for a bit.I don't know any of the people in the movie besides Carmen Electra, Vanessa Minnillo, and Kim Kardashian, but it doesn't matter. They're all horrible, though I think that was the point. The editing is flat out horrible, and possibly blatant continuity errors make this crapfast even crappier than I thought it would be. Now I know that these films are not supposed to be serious at all, but come on, it's film-making 101 that if someone gets a minor facial cut, it should be there in the...",Disaster Movie,"* IsenÃ§Ã£o de responsabilidade: eu sÃ³ assisti esse filme como um acordo condicional. E eu vejo filmes de graÃ§a. Eu nÃ£o seria pego morto dando meu dinheiro suado a esses idiotas. Bem, para explicar a profundidade desse 'filme', eu poderia escrever minha crÃ­tica mais curta de todos os tempos. NÃ£o vÃª este filme. Ã de longe o filme mais estÃºpido, lamenta, preguiÃ§oso e inacreditavelmente UNFUNNY que eu jÃ¡ vi. Ã um desastre total. Mas como o meu Ã³dio por este filme e por outros, se estende muito alÃ©m de uma exibiÃ§Ã£o, acho que vou continuar um pouco. NÃ£o conheÃ§o nenhuma das pessoas do filme alÃ©m de Carmen Electra, Vanessa Minnillo, e Kim Kardashian, mas isso nÃ£o importa. Eles sÃ£o todos horrÃ­veis, embora eu ache que esse seja o ponto. A ediÃ§Ã£o Ã© horrÃ­vel e, possivelmente, erros de continuidade flagrantes tornam essa porcaria ainda mais horrÃ­vel do que eu pensava. Agora eu sei que esses filmes nÃ£o devem ser sÃ©rios, mas vamos lÃ¡, Ã© o cinema 101 que se alguÃ©m f...",disclaimer watched movie conditional agreement see films free not caught dead giving hard earned money idiots well explain depth film write shortest review ever not see movie far stupidest lamest lazy unbelievably unfunny movie ever seen total disaster since hatred movie others like extends far beyond one viewing think go bit not know people movie besides carmen electra vanessa minnillo kim kardashian not matter horrible though think point editing flat horrible possibly blatant continuity errors make crapfast even crappier thought know films not supposed serious come film making someone gets minor facial cut next shot someone gets cut sword blood least cut though since narnia films get away give disaster movie pass jokes thoughtless mindless physical gags obviously take popular movies last year late well including best picture nominees know saddest thing stupid movies not care much money make many cameos sorry ass excuses films taking away jobs actors writers directors truly deserv...
1,1.0,"I am writing this in hopes that this gets put over the previous review of this ""film"". How anyone can find this slop entertaining is completely beyond me. First of all a spoof film entitled ""Disaster Movie"", should indeed be a spoof on disaster films. Now I have seen 1 (yes count them, 1) disaster film being spoofed, that being ""Twister"". How does Juno, Iron Man, Batman, The Hulk, Alvin and the Chipmunks, Amy Winehouse, or Hancock register as Disaster films? Selzterwater and Failburg once again have shown that they lack any sort of writing skill and humor. Having unfortunately been tortured with Date Movie and Epic Movie I know exactly what to expect from these two...no plot, no jokes just bad references and cheaply remade scenes from other films. Someone should have informed them that satire is more than just copy and paste from one film to another, though I shouldn't say that because some of these ac

In [22]:
#Checking for missing value
df.isna().sum()

Ratings           0
Reviews           0
Movies           25
Resenhas          0
Reviews_clean     0
dtype: int64

In [23]:
df['Ratings'].describe()

count    150000.000000
mean          5.500000
std           2.872291
min           1.000000
25%           3.000000
50%           5.500000
75%           8.000000
max          10.000000
Name: Ratings, dtype: float64

In [40]:
df['Reviews_clean'].describe()

count                                                                   150000
unique                                                                  149755
top       story soundtrack dialog graphic reasonableness entertainment overall
freq                                                                        10
Name: Reviews_clean, dtype: object

### Mapping rating data to Binary label:
positive reviews:1 (+ve) if rating >=7 
Negative Reviews: 0 (-ve) if rating <=4 
Neutral Reviews: 2 (neutral) if rating = 5 or 6

In [15]:
df['Label'] = df['Ratings'].apply(lambda x: '1' if x >= 7 else ('0' if x<=4 else '2'))
df['Label'].value_counts()

Label
0    60000
1    60000
2    30000
Name: count, dtype: int64

In [16]:
#Removing Neutral Reviews because now we are going to do sentiment analysis for positive and negative only: 
df=df[df.Label<'2']
data=df[['Reviews_clean','Label']]
print(data['Label'].value_counts())

Label
0    60000
1    60000
Name: count, dtype: int64


In [17]:
#Importing dependencies for feature engineering 
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from prettytable import PrettyTable
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

## Lemmatization:
It basically converts the word to its root word.

In [18]:
# lemmatization of word 
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

## Vectoization with Count Vectorizer and TDIDF Vectorizer with Unigram

Notes:
Here max_feature = 500 which means it takes top 500 most frequent used words from entire Dataset
and makes the encodings.
analyzer = 'word' which means it takes each words as tokens. you can give each character as token also.
min_df =10 means it takes only words that appear in atleast 10 documents. so it filter out very rare words

In [19]:
# Train test split
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
print(train.shape,test.shape)

In [20]:
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)

In [21]:
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray() # It converts word to vector form
x_test_count = countvect.transform(test['Reviews_clean']).toarray()

In [22]:
%%time
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()# It converts word to vector form.
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()

CPU times: total: 38min 9s
Wall time: 39min 39s


In [23]:
y_train = train['Label']
y_test = test['Label']

## Feature Importance with Logistic Regression and Count Vectorizer for unigram

In [ ]:
%%time
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]

In [25]:
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|      able      |  0.10051132271922628   |
|   absolutely   |  0.047591182075346045  |
|      act       |  -0.15708819921594505  |
|     acting     |  -0.16709427152003606  |
|     action     |  0.26787148017490114   |
|     actor      |  -0.12909189686051942  |
|    actress     |  -0.14910267903353383  |
|    actually    | -0.052672220233663795  |
|      add       |  0.025939151568154822  |
|     adult      |  0.059908425091800346  |
|      age       |  0.10016341042002383   |
|     alien      |  -0.07112947595725591  |
|     almost     | -0.014782888621588886  |
|     along      |  0.12421208662818702   |
|    already     |  -0.20326876281200226  |
|      also      |  0.09806641664732631   |
|    although    |  0.19841679553603878   |
|     always     |  0.20158659751710958   |
|    amazing     |   0.8423592465569725   |
|    american    |   0.054676832

## Feature Importance with TFIDF vectorizer and Logistic Regression with Unigram

In [26]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+----------------+-----------------------+
|    Feature     |         Score         |
+----------------+-----------------------+
|      able      |   0.3886384941416218  |
|   absolutely   |   0.4707201948406383  |
|      act       |  -1.0474963284894492  |
|     acting     |  -1.6658494140647735  |
|     action     |   2.649249094364551   |
|     actor      |  -1.3608978523877509  |
|    actress     |  -1.0142257182662608  |
|    actually    |  -0.4769293109879931  |
|      add       |  0.13035725765251027  |
|     adult      |   0.6919000679359252  |
|      age       |   0.765501922306687   |
|     alien      |  -0.3348929531223187  |
|     almost     |  -0.28854515822910837 |
|     along      |   0.7504371072734287  |
|    already     |  -1.1445189484724636  |
|      also      |   1.224171897977061   |
|    although    |   1.2307145820397944  |
|     always     |   1.7096080356819434  |
|    amazing     |   4.791003637030143   |
|    american    |  0.46326859160761313  |
|    annoyi

## Vectorization with Count Vectorizer and TDIDF Vectorizer with Bigram

In [27]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)

In [28]:
%%time
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()

CPU times: total: 37min 9s
Wall time: 38min 37s


In [29]:
%%time
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()

CPU times: total: 37min 19s
Wall time: 38min 21s


In [30]:
y_train = train['Label']
y_test = test['Label']

## Feature Importance with Logistic Regression and Count Vectorizer with Bigram

In [31]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-----------------------+------------------------+
|        Feature        |         Score          |
+-----------------------+------------------------+
|       acting bad      |  -1.3592826414968588   |
|      acting good      |   0.6547550628805627   |
|       acting not      |  -0.46782061995038077  |
|      action film      |   0.3699274125482786   |
|      action movie     |   0.2910595529217947   |
|     action packed     |   1.3386783052943638   |
|      action scene     |  0.17518973669870858   |
|    action sequence    |   0.1015169625215072   |
|     actor actress     |  -0.12557338892683603  |
|       actor not       |  -0.39186080624001013  |
|      almost every     |  -0.13260075331280596  |
|       along way       |   0.4908860425707029   |
|        also not       |  -0.1397495696846917   |
|      although not     |  0.49910230925009536   |
|     anything else     |  -0.3232091756465047   |
|        b movie        |  -0.09071784029430811  |
|       bad acting      |  -1.0

## Feature Importance with Logistic Regression and TFIDF Vectorizer with Bigram

In [33]:
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=50:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-----------------------+----------------------+
|        Feature        |        Score         |
+-----------------------+----------------------+
|       acting bad      | -2.9871760534321985  |
|      acting good      |  1.4431054145457127  |
|       acting not      | -1.2744698745264176  |
|      action film      |  1.223715087435864   |
|      action movie     |  0.8443556589356428  |
|     action packed     |  3.0238449287615943  |
|      action scene     | 0.47004358211870106  |
|    action sequence    |  0.2964462639054979  |
|     actor actress     | -0.2615181920368176  |
|       actor not       | -0.9344034381743002  |
|      almost every     | -0.29476710752147317 |
|       along way       |  1.0753056686648286  |
|        also not       | -0.2909297593308741  |
|      although not     |  1.104396976989652   |
|     anything else     | -0.7966422564307266  |
|        b movie        | -0.42286159751415836 |
|       bad acting      |  -2.799472965808038  |
|        bad bad    

## Vectorization with Count Vectorizer and TFIDF Vectorizer with Trigram

In [34]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", ngram_range=(3,3), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", ngram_range=(3,3),min_df=10,max_features=500)

In [35]:
%%time
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()

CPU times: total: 42.6 s
Wall time: 45 s


In [36]:
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()

In [37]:
y_train = train['Label']
y_test = test['Label']

## Feature Importance with Logistic Regression and Count Vectorizer with Trigram

In [38]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------------+-----------------------+
|          Feature          |         Score         |
+---------------------------+-----------------------+
|       acting not bad      |  -0.4350002951764665  |
|      acting not good      |   -1.092491139760708  |
|      acting not great     |  -0.5343838708550173  |
|     acting pretty good    |   0.8970716688429763  |
|   acting special effects  |  -0.5546971013254138  |
|      acting top notch     |   1.8134221851772077  |
|      action movie not     |   0.9758552898229472  |
|     action takes place    |  0.13058684590424677  |
|      actors good job      |   0.7461475204539747  |
|      actually not bad     |  -0.8048817533890187  |
|    actually pretty good   |   0.3274731661477579  |
|    actually quite good    |  -0.17655777924965885 |
|     almost every scene    |  -0.3131896882395349  |
|     avoid like plague     |   -2.566575627262404  |
|       bad acting bad      |   -2.268995746261501  |
|        bad bad bad        

## Feature Importance with Logistic Regression and TFIDF Vectorizer with Trigram

In [39]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+---------------------------+------------------------+
|          Feature          |         Score          |
+---------------------------+------------------------+
|       acting not bad      |  -0.5755422668630829   |
|      acting not good      |   -1.336979962084483   |
|      acting not great     |  -0.6959513888560159   |
|     acting pretty good    |   1.0255330536291545   |
|   acting special effects  |  -0.5691491757943977   |
|      acting top notch     |   2.0385971213626526   |
|      action movie not     |   1.097006739763075    |
|     action takes place    |  0.18564182057328696   |
|      actors good job      |   0.8897952893895097   |
|      actually not bad     |  -0.9870803476671267   |
|    actually pretty good   |   0.3888771661223337   |
|    actually quite good    |  -0.18096844462849748  |
|     almost every scene    |  -0.3955489430127699   |
|     avoid like plague     |  -2.7538639572657124   |
|       bad acting bad      |   -2.817745457436599   |
|        b